In [ ]:
import numpy as np
import pandas as pd
import copy
pd.set_option('display.max_columns', None)

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from itertools import chain

import requests
base_url_fir = "https://sports.core.api.espn.com"
base_url_sec = 'https://site.web.api.espn.com'
queries_for_ids = {
    "athletes": base_url_fir + "/v2/sports/football/leagues/nfl/athletes?limit=1000&active=true",
    "teams": base_url_fir + "/v2/sports/football/leagues/nfl/teams?limit=32",
    "positions": base_url_fir +"/v2/sports/football/leagues/nfl/positions?limit=75",
    "venues": base_url_fir +"/v2/sports/football/leagues/nfl/venues?limit=700",
}

In [ ]:
def get_full_response_list(base):
    response = requests.get(base).json()
    response_list = []

    if 'pageIndex' in response.keys():
        for i in range(response['pageCount']):
            response_list.append(requests.get(base + f'&page={i + 1}').json()['items'])
        response_list = [x['$ref'] for x in list(chain.from_iterable(response_list))]
        # response_list_cl = list(chain.from_iterable(response_list))
    else:
        response_list = [x['$ref'] for x in response['items']]
        # response_list_cl = response['items']


    return response_list


def del_cell_ref_field(cell):
    cell = copy.deepcopy(cell)
    if (isinstance(cell, dict)) and ('$ref' in cell.keys()):
        del cell['$ref']
        return cell
    return cell


def request_table_data(df_name, source_column, target_column):
    df_name[target_column] = df_name[source_column].apply(lambda x: x if pd.isnull(x) else del_cell_ref_field(requests.get(x).json()))


def process_link_columns(df_name, source_column_list):
    for i in source_column_list:
        request_table_data(df_name, i, 'loaded_from_' + i)
    df_name.drop(columns=source_column_list, inplace=True)


def check_dict_link(cell):
    if isinstance(cell, dict):
        return (cell == requests.get(cell['$ref']).json())
    return False


def unwrap_data(df, column_name):

    if df[column_name].apply(lambda x: isinstance(x, list)).any():

            columns_list = [f'col_{x}_from_{column_name}' for x in range(df[column_name].apply(lambda x: len(x) if isinstance(x, list) else 0).max())]
            sup_df = pd.DataFrame(columns=columns_list, index=range(df.shape[0]))
            for row, field in enumerate(df[column_name]):
                if not isinstance(field, list):
                    field = [field]
                for num, block in enumerate(field):
                    sup_df.at[row, f'col_{num}_from_{column_name}'] = block


    elif df[column_name].notna().any():

        if (df[column_name].apply(lambda x: ((isinstance(x, dict))) and ('$ref' in x.keys())).any()) and (df[column_name].apply(lambda x: check_dict_link(x)).any()):
            df[column_name] = df[column_name].apply(lambda x: del_cell_ref_field(x))

        col_names = set()

        for i in df[column_name]:
            if isinstance(i, dict):
                col_names = col_names.union(i.keys())

        col_names = list(col_names)


        sup_df = pd.DataFrame(columns=[(x + '_from_' + column_name) for x in col_names],
                              index=range(df.shape[0]))
        for i, j in enumerate(df[column_name]):
            if not isinstance(j, dict) or pd.isnull(j):
                for keys in col_names:
                    sup_df.at[i, keys + '_from_' + column_name] = j
            else:
                for keys, values in j.items():
                    sup_df.at[i, keys + '_from_' + column_name] = values

    else:
        return pd.DataFrame()
    sup_df.index = df.index
    return sup_df


def unwrap_list_columns(df_name, source_column_list):
    unwrapped_list = []
    for i in source_column_list:
        df_name = pd.concat([df_name, unwrap_data(df_name, i)], axis = 1)
        unwrapped_list.append(i)
    df_name.drop(columns=source_column_list, inplace=True)
    if not [x for x in source_column_list if x not in unwrapped_list]:
        print('Unwrapped successful')
    return df_name


def get_parent_position_id(base):
    if pd.isna(base):
        return np.nan
    result = requests.get(base).json()
    if ('id' in result.keys()):
        return result['id']
    return np.nan


# Функция для получения ID атлета
def get_athlete_id(base):
    if pd.isna(base):
        return np.nan
    answer = requests.get(base).json()
    if ('id' in answer.keys()):
        return answer['id']
    return np.nan


# Функция для получения года и имени сезона для идентификации команды
def get_season_start_and_team_id(base):
    if pd.isna(base):
        return np.nan, np.nan
    answer = requests.get(base).json()
    keys = answer.keys()
    if 'id' in keys:
        team_id = answer['id']
    else:
        team_id = np.nan
    if 'groups' not in keys:
        return team_id, np.nan
    if len(answer['groups'].keys()) != 1:
        return team_id, np.nan
    result = answer['groups']['$ref']
    result = requests.get(result).json()['season']['$ref']
    result = requests.get(result).json()
    return team_id, result['startDate']


def get_season_start(base):
    if pd.isna(base):
        return np.nan
    result = requests.get(base).json()
    if ('startDate' in result.keys()):
        return result['startDate']
    return np.nan

In [ ]:
athletes_df = pd.DataFrame(get_full_response_list('http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes?lang=en&region=us&limit=1000'),
                           columns=['$ref_on_athletes'])

In [ ]:
chunks_list = []
chunks_number = 250
for i in range(chunks_number):
    chunks_list.append(athletes_df.iloc[range((athletes_df.shape[0] // chunks_number) * i,
                                              (athletes_df.shape[0] // chunks_number) * (i + 1)), :].copy())
chunks_list.append(athletes_df.iloc[range((athletes_df.shape[0] // chunks_number) * chunks_number, athletes_df.shape[0]), :].copy())

In [ ]:
def completely_preprocess_data(chunk):
    process_link_columns(chunk, ['$ref_on_athletes'])
    chunk = unwrap_list_columns(chunk, ['loaded_from_$ref_on_athletes'])

    while [x for x in chunk.columns if (chunk[x].apply(lambda x: isinstance(x, (dict, list)))).any()]:
        chunk = unwrap_list_columns(chunk,
                                 [x for x in chunk.columns if (chunk[x].apply(lambda x: isinstance(x, (dict, list)))).any()])

    for i in [x for x in chunk.columns if x.startswith('$ref_from_team')]:
        values_tuple = chunk[i].apply(get_season_start_and_team_id)
        chunk['team_id_from_' + i] = [x[0] for x in values_tuple]
        chunk['season_start_from_' + i] = [x[1] for x in values_tuple]
    chunk.drop(columns=[x for x in chunk.columns if x.startswith('$ref_from_team')], inplace=True)

    process_link_columns(chunk, [x for x in chunk.columns if x.startswith('$ref')])

    while [x for x in chunk.columns if (chunk[x].apply(lambda x: isinstance(x, (dict, list)))).any()]:
        chunk = unwrap_list_columns(chunk,
                                 [x for x in chunk.columns if (chunk[x].apply(lambda x: isinstance(x, (dict, list)))).any()])

    for i in [x for x in chunk.columns if x.startswith('$ref_from_team')]:
        values_tuple = chunk[i].apply(get_season_start_and_team_id)
        chunk['team_id_from_' + i] = [x[0] for x in values_tuple]
        chunk['season_start_from_' + i] = [x[1] for x in values_tuple]
    chunk.drop(columns=[x for x in chunk.columns if x.startswith('$ref_from_team')], inplace=True)

    for i in [x for x in chunk.columns if (x.startswith('$ref_from_athlete') or (x.startswith('$ref_from_proAthlete')))]:
        chunk['athlete_id_from_' + i] = chunk[i].apply(get_athlete_id)
    chunk.drop(columns=[x for x in chunk.columns if (x.startswith('$ref_from_athlete') or (x.startswith('$ref_from_proAthlete')))], inplace=True)

    for i in [x for x in chunk.columns if x.startswith('$ref_from_season')]:
        chunk['season_start_from_' + i] = chunk[i].apply(get_season_start)
    chunk.drop(columns=[x for x in chunk.columns if x.startswith('$ref_from_season')], inplace=True)

    for i in [x for x in chunk.columns if x.startswith('$ref_from_parent')]:
        chunk['parent_position_id_from_' + i] = chunk[i].apply(get_parent_position_id)
    chunk.drop(columns=[x for x in chunk.columns if x.startswith('$ref_from_parent')], inplace=True)

    process_link_columns(chunk, [x for x in chunk.columns if x.startswith('$ref')])

    while [x for x in chunk.columns if (chunk[x].apply(lambda x: isinstance(x, (dict, list)))).any()]:
        chunk = unwrap_list_columns(chunk,
                                 [x for x in chunk.columns if (chunk[x].apply(lambda x: isinstance(x, (dict, list)))).any()])
    return chunk

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

2

In [ ]:
(pd.read_csv('/content/drive/MyDrive/BidData/data/ChunkProcessAttempt/Probe_210.csv')).shape

<ipython-input-7-faebb4b9422e>:1: DtypeWarning: Columns (20,28,29,30,140,144,145,153,154,163,164,165,172,182,183,291,296,299,302,305,312,313,314,315,387,388,389,390,391,394,395,396,397,398,401,402,403,404,405,408,409,410,411,412,415,416,417,418,419,422,423,424,425,426,562,814,1017,1142,1144,1145,1146,1147,1149,1151,1152,1153,1154,1156,1158,1159,1160,1161,1163,1165,1166,1167,1168,1171,1172,1173,1174,1175,1178,1179,1180,1181,1182,1185,1186,1187,1188,1189,1192,1193,1194,1195,1196,1199,1200,1201,1202,1203,1206,1207,1208,1209,1210,1422,1424,1425,1426,1427,1429,1431,1432,1433,1434,1436,1438,1439,1440,1441,1443,1445,1446,1447,1448,1471,1473,1474,1475,1476,1478,1480,1481,1482,1483,1485,1487,1488,1489,1490,1492,1494,1495,1496,1497,1499,1501,1502,1503,1504,1506,1508,1509,1510,1511,1513,1515,1516,1517,1518,1520,1522,1523,1524,1525,1527,1529,1530,1531,1532,1534,1536,1537,1538,1539,1541,1543,1544,1545,1546,1548,1550,1551,1552,1553,1555,1557,1558,1559,1560,1801,1802,1803,1804,1805,1808,1809,1810,181

(71, 35794)

In [ ]:
def multy_proc_preprocess(chunk_num):
    res_df = completely_preprocess_data(chunks_list[chunk_num])
    print(f'Chunk {chunk_num} processed')
    # saved_df = pd.read_csv(f'/content/drive/MyDrive/BidData/data/ChunkProcessAttempt/Probe{chunk_num}.csv')
    # df_to_upload = pd.concat([saved_df, res_df], axis=0)
    res_df.to_csv(f'/content/drive/MyDrive/BidData/data/Athletes_Chunks_Only/Probe_{chunk_num}.csv')
    print('Probe file uploaded')

In [ ]:
# #13,
# for i in range(21, 30, 2):
#     chunks_num_list = [i, i+1]
#     processes = [multiprocessing.Process(target=multy_proc_preprocess, args=[num]) for num in chunks_num_list]

#     for process in processes:
#         process.start()

#     # дожидаемся выполнение
#     for process in processes:
#         process.join()

In [ ]:
import os
content = os.listdir('/content/drive/MyDrive/BidData/data/Athletes_Chunks_Only')
content = [int(x.split('_')[1].split('.')[0]) for x in content]

In [ ]:
upload_chunks = [x for x in range(11, 81) if x not in content]

In [ ]:
len(upload_chunks)//2

1

In [ ]:
for i in range(0, len(upload_chunks)//2, 2):
    # chunks_num_list = [upload_chunks[i], upload_chunks[i+1]]
    # print(f'Process chunk {upload_chunks[i]} and chunk {upload_chunks[i+1]}')
    # processes = [multiprocessing.Process(target=multy_proc_preprocess, args=[num]) for num in chunks_num_list]

    chunks_num_list = [45]
    # print(f'Process chunk {upload_chunks[i]} and chunk {upload_chunks[i+1]}')
    print(f'Process chunk {45}')
    processes = [multiprocessing.Process(target=multy_proc_preprocess, args=[num]) for num in chunks_num_list]

    for process in processes:
        process.start()

    # дожидаемся выполнение
    for process in processes:
        process.join()

Process chunk 45
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Unwrapped successful
Chunk 45 processed
Probe file uploaded


In [ ]:
processes = [multiprocessing.Process(target=multy_proc_preprocess, args=[num]) for num in chunks_num_list]

In [ ]:
for process in processes:
    process.start()

# дожидаемся выполнение
for process in processes:
    process.join()

In [ ]:
# is_success = completely_preprocess_data(chunks_list[800])

In [ ]:
# empty_pd = pd.DataFrame()

In [ ]:
# empty_pd.to_csv('/content/drive/MyDrive/BidData/data/ChunkProcessAttempt/GeneralFile.csv')

In [ ]:
# for i in range(3):
#     res_df = completely_preprocess_data(chunks_list[i])
#     print(f'Chunk {i} processed')
#     saved_df = pd.read_csv('/content/drive/MyDrive/BidData/data/ChunkProcessAttempt/GeneralFile.csv')
#     df_to_upload = pd.concat([saved_df, res_df], axis=0)
#     df_to_upload.to_csv('/content/drive/MyDrive/BidData/data/ChunkProcessAttempt/GeneralFile.csv')
#     print('General file updated')

In [ ]:
saved_df = pd.read_csv('/content/drive/MyDrive/BidData/data/ChunkProcessAttempt/GeneralFile.csv')

<ipython-input-10-81bc79c7b237>:1: DtypeWarning: Columns (7,140,143,146,150,151,152,174,175,261,262,263,264,266,267,268,269,271,272,273,274,276,277,278,279,282,283,286,293,296,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,364,365,367,368,370,371,372,374,375,377,378,379,381,382,384,385,386,388,389,391,392,393,395,396,398,399,400,402,403,405,406,407,409,410,412,413,414,416,417,419,420,421,423,424,426,565,817,1020,1041,1141,1142,1144,1145,1147,1148,1149,1151,1152,1154,1155,1156,1158,1159,1161,1162,1163,1165,1166,1168,1169,1170,1172,1173,1175,1176,1177,1179,1180,1182,1183,1184,1186,1187,1189,1190,1191,1193,1194,1196,1197,1198,1200,1201,1203,1204,1205,1207,1208,1210,1211,1212,1214,1215,1217,1218,1219,1221,1222,1224,1225,1226,1228,1229,1231,1232,1233,1235,1236,1238,1239,1240,1242,1243,1245,1246,1247,1249,1250,1252,1253,1254,1256,1257,1259,1260,1261,1263,1264,1266,1267,1268,1270,1271,1273,1274,1275,1277,1278,1280,1281,1282,1284,1285,1287,1288,1289,1291,1292,1294,1295,1296,12

In [ ]:
saved_df.shape

(213, 54042)